# Invoke the Production Center v2 WPS services using Python

## Setup the Access Credentials

### Username and Login Token
Username and Login Token provide access to the WPS endpoint. The Login Token can be permanent or have a temporal validity. It can be generate from the [User Profile](https://docs.terradue.com/ellip/accounts/myaccount/profile.html) page. 

#### Note
_In case the Login Token has a temporal validity, it can also be stored within this document._

In [1]:
username = 'crossi'

# this token has a short temporal validity
login_token = 'da39a3ee5e6b4b0d3255bfef95601890afd80709'

wps_credentials = username + ':' + login_token

### API Key
API key provides access to the [Data Agency](https://docs.terradue.com/ellip/core/da/index.html), for the stage-out of the processing results.

#### Note
_The API Key is permanent, so it cannot be stored within this document._

In [2]:
import getpass
api_key  = getpass.getpass(prompt='API key: ', stream=None)

## Inspect the OGC WPS capabilities

In [3]:
from owslib.wps import WebProcessingService
from owslib.wps import printInputOutput

In [4]:
wps_url = "https://wps-pc.terradue.com/zoo/?"
wps = WebProcessingService(wps_url, verbose=False, skip_caps=True, headers={'Authorization': wps_credentials})

In [5]:
wps.getcapabilities()

In [6]:
for operation in wps.operations:
    print(operation.name)

GetCapabilities
DescribeProcess
Execute


In [7]:
for process in wps.processes:
    print(process.identifier)

GetStatus
jobcatalog
i5188c3b27e04b30c5f1186cb525c0fb02070e687


## Select a processing service

In [8]:
process_id = 'i5188c3b27e04b30c5f1186cb525c0fb02070e687'

## Inspect the OGC WPS process information and inputs

In [9]:
process = wps.describeprocess(process_id)

In [10]:
process.title

'Full Resolution Rasterization'

In [11]:
process.abstract

'This service allows to generate full resolution RGB (optical) and single-channel (SAR) GeoTiff images from EO data. The service supports EO data from the following missions: ALOS-2 (L1.5), GF-2 (L2A), Kanopus-V IK (MSS_PSS) , Kompsat-2/-3 (L1G PAN_MS), Kompsat-5 (L1D GTC), Landsat 8 (L1TP,L1T,L1GT), Pleiades 1 (ORTHO), Radarsat-2 (SGF), Rapideye (L3A ORTHO), Resurs-P (KSHMSA), Sentinel-1 (L1 GRD), Sentinel-2 (L1C, L2A), Sentinel-3 (L1 OLCI ERR/EFR, L1 SLSTR RBT), SPOT-6/7 (L3 ORTHO), Terrasar-X (L1B EEC), UK-DMC 2 (L1T), VRSS1 (L2B). The tutorial can be found at: https://terradue.github.io/doc-tep-geohazards/tutorials/rss_full_resolution_rasterization.html'

In [12]:
for input in process.dataInputs:
    printInputOutput(input)
    print('***')

 identifier=data_reference, title=Data reference, abstract=Define the data product reference to be browsed in full resolution, data type=string
 Any value allowed
 Default Value: None 
 minOccurs=1, maxOccurs=9999
***
 identifier=_T2Username, title=User name, abstract=Valid usernames, data type=string
 Any value allowed
 Default Value: None 
 minOccurs=1, maxOccurs=1
***
 identifier=_T2Apikey, title=User Api key, abstract=Valid user's Api key, data type=string
 Any value allowed
 Default Value: null 
 minOccurs=1, maxOccurs=1
***
 identifier=_T2JobInfoFeed, title=Job Info Atom Feed, abstract=Atom Feed containing information about the WPS job, data type=string
 Any value allowed
 Default Value: null 
 minOccurs=0, maxOccurs=1
***
 identifier=_T2ResultsAnalysis, title=Results Analysis, abstract=Terradue results analysis system, data type=string
 Allowed Value: skip
 Allowed Value: basic
 Allowed Value: intermediate
 Allowed Value: advanced
 Allowed Value: extended
 Default Value: basic 


Prepare the execution request

In [13]:
inputs = []
inputs.append(('data_reference','https://catalog.terradue.com/sentinel2/search?format=json&uid=S2A_MSIL2A_20190908T235241_N0213_R130_T56JNN_20190909T015043'))
inputs.append(('_T2Username', username))
inputs.append(('_T2Apikey', api_key))
inputs.append(('_T2ResultsAnalysis','advanced'))

outputs = []
outputs.append(('result_osd',False))
outputs.append(('t2_logs',False))

## Execute and monitor the request

In [14]:
execution = wps.execute(process_id, inputs, outputs)

In [15]:
# This is a workaround because the provided statusLocation protocol is http, which is not correct
statusLocation = execution.statusLocation.replace('http://','https://')

In [21]:
statusLocation

'https://wps-pc.terradue.com/zoo-bin/zoo_loader.cgi?request=Execute&service=WPS&version=1.0.0&Identifier=GetStatus&DataInputs=sid=9254bcf4-806e-11ea-b283-0a580a80064b&RawDataOutput=Result'

In [22]:
execution.getStatus()

'ProcessAccepted'

In [28]:
execution.checkStatus(url=statusLocation)

In [26]:
execution.getStatus()

'ProcessAccepted'

In [38]:
import requests
import xml.dom.minidom

In [40]:
response = requests.get(statusLocation)

dom = xml.dom.minidom.parseString(response.content)
print(dom.toprettyxml())

<?xml version="1.0" ?>
<wps:ExecuteResponse xmlns:ows="http://www.opengis.net/ows/1.1" xmlns:wps="http://www.opengis.net/wps/1.0.0" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/wps/1.0.0 http://schemas.opengis.net/wps/1.0.0/wpsExecute_response.xsd" service="WPS" version="1.0.0" xml:lang="en-US" serviceInstance="http://wps-pc.terradue.com/zoo-bin/zoo_loader.cgi" statusLocation="http://wps-pc.terradue.com/zoo-bin/zoo_loader.cgi?request=Execute&amp;service=WPS&amp;version=1.0.0&amp;Identifier=GetStatus&amp;DataInputs=sid=9254bcf4-806e-11ea-b283-0a580a80064b&amp;RawDataOutput=Result">
	
  
	<wps:Process wps:processVersion="1.0.0">
		
    
		<ows:Identifier>i5188c3b27e04b30c5f1186cb525c0fb02070e687</ows:Identifier>
		
    
		<ows:Title>Full Resolution Rasterization</ows:Title>
		
    
		<ows:Abstract>This service allows to generate full resolution RGB (optical) and single-channel (SAR) GeoTiff ima

### Notes

There is an issue with the [checkStatus](https://github.com/geopython/OWSLib/blob/45cbc2324ca372ea7745350e47a4cc90bdf21b60/owslib/wps.py#L790) method (or with [monitorExecution](https://github.com/geopython/OWSLib/blob/45cbc2324ca372ea7745350e47a4cc90bdf21b60/owslib/wps.py#L1886)) which does not update the status with this WPS server. It might be caused by the lack of redirection from HTTP to HTTPS of the statusLocation URL (see **Redirect HTTP to HTTPS for wps-pc.terradue.com** in [Production Center v2 - Recommendations from TTO](https://docs.google.com/document/d/1ks1Wz4Ckn567OhmuZZqiLXwdcJIQCCTbVYruAAMEtyE/edit#heading=h.uwl0uv44if45). A further investigation is required for this client code.